In [1]:
!pip install docx2txt langchain-community
!pip install langchain-text-splitters
!pip install langchain-chroma

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,     # 각 청크의 최대 문자 수
    chunk_overlap=200    # 청크 간 겹치는 문자 수
)

loader = Docx2txtLoader('./tax.docx')
tax_documents = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [5]:
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=tax_documents, embedding=embedding, collection_name='chroma-tax', persist_directory='./chroma')

In [9]:
query = '연봉 5천만원 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [12]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

[Question]
{query}
"""

ai_message = llm.invoke(prompt)
ai_message.content

'안녕하세요, 최고의 한국 소득세 전문가입니다.\n\n연봉 5천만원 직장인의 소득세를 정확히 계산하기 위해서는 몇 가지 추가 정보가 필요합니다. 세금은 단순히 연봉에 세율을 곱하는 것이 아니라, 다양한 소득공제와 세액공제를 거쳐 계산되기 때문입니다.\n\n[Context] 문서에서 소득의 구분(제4조)과 공동소유 자산의 납세의무(제2조), 신탁재산 귀속 소득(제2조의3), 그리고 특별소득공제의 일환으로 장기주택저당차입금 이자상환액 소득공제(제52조) 및 부양가족의 범위(제53조) 등에 대한 규정은 제시되어 있으나, 구체적인 근로소득공제율, 인적공제액, 세액공제 항목 및 세율 등 소득세 계산에 필요한 모든 정보는 포함되어 있지 않습니다.\n\n하지만 전문가로서 소득세 계산의 일반적인 흐름과 필요한 정보는 다음과 같습니다.\n\n**소득세 계산 과정 (근로소득자의 경우):**\n\n1.  **총급여액 확인:** 연봉 5천만원이 총급여액이 됩니다.\n2.  **근로소득공제 적용:** 총급여액에서 소득세법에 따른 근로소득공제를 차감합니다. 이 공제는 총급여액 구간별로 정해진 금액 또는 비율을 적용합니다.\n3.  **근로소득금액 산출:** (총급여액 - 근로소득공제) = 근로소득금액\n4.  **종합소득공제 적용:** 근로소득금액에서 인적공제(본인, 배우자, 부양가족), 연금보험료 공제(국민연금 등), 건강보험료 공제, 주택자금 공제, 신용카드 등 사용액 공제 등 다양한 소득공제를 차감합니다.\n5.  **과세표준 산출:** (근로소득금액 - 종합소득공제) = 과세표준\n6.  **산출세액 계산:** 과세표준에 소득세율(누진세율)을 적용하여 산출세액을 계산합니다.\n7.  **세액공제 적용:** 산출세액에서 근로소득세액공제, 자녀세액공제, 연금계좌세액공제, 의료비/교육비/기부금 세액공제 등 각종 세액공제를 차감합니다.\n8.  **결정세액 산출:** (산출세액 - 세액공제) = 최종 납부할 소득세액\n\n**연봉 5천만원 직장인의 소득세 계산을 위해 필요한 추가 정보